# **Preparación de los datos**
A continuación, se realiza una preparación y limpieza más minuciosa de los datos a usar. Para esto es necesario identificar datos duplicados, faltantes, aberrantes y atípicos. Asimismo, asegurarse de que los datos queden en un formato que permita su posterior análisis para la etapa de modelamiento.

## **1. Importación de los datos**
Estos se encuentran en formato `.csv`, se importa y se manejan como DataFrame con la libreria `pandas`.

In [1]:
import pandas as pd

df = pd.read_csv("incident_event_log.csv")
print(df.head())

       number incident_state  active  reassignment_count  reopen_count  \
0  INC0000045            New    True                   0             0   
1  INC0000045       Resolved    True                   0             0   
2  INC0000045       Resolved    True                   0             0   
3  INC0000045         Closed   False                   0             0   
4  INC0000047            New    True                   0             0   

   sys_mod_count  made_sla    caller_id       opened_by        opened_at  ...  \
0              0      True  Caller 2403    Opened by  8  29/2/2016 01:16  ...   
1              2      True  Caller 2403    Opened by  8  29/2/2016 01:16  ...   
2              3      True  Caller 2403    Opened by  8  29/2/2016 01:16  ...   
3              4      True  Caller 2403    Opened by  8  29/2/2016 01:16  ...   
4              0      True  Caller 2403  Opened by  397  29/2/2016 04:40  ...   

  u_priority_confirmation         notify problem_id rfc vendor cause

## **2. Eliminación de variables irrelevantes**
La idea es limpiar columnas que no aportan o que pueden generar ruido, y quedarnos con variables que explican **tiempo de resolución, efectividad de los asesores y características del incidente.** Debido a la naturaleza del problema y las preguntas de negocio que buscamos resolver, se asume nula o poca relevancia para las siguientes varibles por las siguientes razones:

`active, incident_state, contact_type`→ prácticamente el 100% de los datos son iguales entonces estas variables no aportan mucho.

`opened_by, sys_created_by, sys_updated_by, assigned_to, resolved_by` → son IDs individuales, generan demasiada cardinalidad (miles de valores distintos) y no permiten análisis útil a este nivel. En todo caso, puede quedarse assignment_group porque agrupa personas.

`problem_id, rfc, vendor, caused_by, close_code, cmdb_ci` → identificadores específicos sin valor analítico general.

`sys_updated_at, sys_created_at` → se pueden transformar en duraciones, pero como columnas crudas son poco manejables.

`notify` → suele ser más de control interno que causa del SLA. Además, el 99% de los datos son `"no notify"`.

In [2]:
df = df.drop(columns=["active","incident_state","opened_by","sys_created_by", "contact_type",
                      "sys_updated_by","resolved_by","contact_type","problem_id",
                      "rfc","vendor","caused_by","closed_code","cmdb_ci",
                      "sys_created_at","notify", "u_priority_confirmation"])
df.head()

,number,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_at,sys_updated_at,location,category,subcategory,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,resolved_at,closed_at
0,INC0000045,0,0,0,True,Caller 2403,29/2/2016 01:16,29/2/2016 01:23,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,0,0,2,True,Caller 2403,29/2/2016 01:16,29/2/2016 08:53,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,0,0,3,True,Caller 2403,29/2/2016 01:16,29/2/2016 11:29,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,0,0,4,True,Caller 2403,29/2/2016 01:16,5/3/2016 12:00,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,?,True,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,0,0,0,True,Caller 2403,29/2/2016 04:40,29/2/2016 04:57,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,1/3/2016 09:52,6/3/2016 10:00


## **3. Preparación de los datos**
Se exploran los datos disponibles y se realiza una limpieza cuidadosa identificando datos duplicados, faltantes, aberrantes y atípicos. Asímismo, es importante asegurarnos de que los datos queden en un formato que permita su posterior análisis, incluso para la etapa de modelamiento. 

**Nota:** se hizo necesario convertir los valores de texto `"?"` a nulo dado que así podemos manipular más facilmente estos datos faltantes.

In [3]:
# Ver si hay duplicados
print("¿Hay duplicados?:", df.duplicated().any())  # True si existe al menos un duplicado
print("Número de filas duplicadas:", df.duplicated().sum())

df = df.drop_duplicates()

# Ver si hay nulos
print("\nCantidad de nulos por columna:")
df.replace('?', pd.NA, inplace=True)
print(df.isnull().sum())

¿Hay duplicados?: False
Número de filas duplicadas: 0

Cantidad de nulos por columna:
number                    0
reassignment_count        0
reopen_count              0
sys_mod_count             0
made_sla                  0
caller_id                29
opened_at                 0
sys_updated_at            0
location                 76
category                 78
subcategory             111
u_symptom             32964
impact                    0
urgency                   0
priority                  0
assignment_group      14213
assigned_to           27496
knowledge                 0
resolved_at            3141
closed_at                 0
dtype: int64


`df.isnull().sum()` nos dice que existen nulos en varibles como `caller_id, location, category, subcategory, u_symptom, assignment_group, assigned_to`, estos nulos no representan una amenaza al tratarse de identificadores o nombres. Sin embargo, la variable `resolved_at` que nos indica la fecha de resolución del incidente, sí representa un problema al tratar de manipular los datos más adelante, ya que causa problemas al tratar esta columna como datos tipo `datetime`. Por esta razón, se decidió asignar la fecha de cierre para los inicidentes sin fecha de resolución.

In [4]:
# Resolver que hacer con las filas con celdas nulas. Sugerencia: llenar resolved_at con closed_at
df.fillna({'resolved_at': df['closed_at']}, inplace=True)

# Formatear las columnas que corresponden a fechas
df["opened_at"] = pd.to_datetime(df["opened_at"], format="%d/%m/%Y %H:%M")
df["resolved_at"] = pd.to_datetime(df["resolved_at"], format="%d/%m/%Y %H:%M")
df["closed_at"] = pd.to_datetime(df["closed_at"], format="%d/%m/%Y %H:%M")

Asimismo, se hizo importante construir un DataFrame aparte que incluyera los incidentes una sola vez, esto para calcular métricas como el **tiempo de resolución**. Esto evita que los datos se vean duplicados por la cantidad de registros que se hacen cada vez que el incidente recibe una actualización.

In [5]:
# Crear un único registro por incidente
df_unicos = df.drop_duplicates(subset="number", keep="last").copy()

# Crear variable de duración en horas
df_unicos["resolution_time"] = (df_unicos["resolved_at"] - df_unicos["opened_at"]).dt.total_seconds() / 3600 / 24 

df_unicos.head(10) 
#print(df_unicos.shape)

,number,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_at,sys_updated_at,location,category,...,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,resolved_at,closed_at,resolution_time
3,INC0000045,0,0,4,True,Caller 2403,2016-02-29 01:16:00,5/3/2016 12:00,Location 143,Category 55,...,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,<NA>,True,2016-02-29 11:29:00,2016-03-05 12:00:00,0.425694
12,INC0000047,1,0,8,True,Caller 2403,2016-02-29 04:40:00,6/3/2016 10:00,Location 165,Category 40,...,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 24,Resolver 89,True,2016-03-01 09:52:00,2016-03-06 10:00:00,1.216667
19,INC0000057,0,0,6,True,Caller 4416,2016-02-29 06:10:00,6/3/2016 03:00,Location 204,Category 20,...,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 6,True,2016-03-01 02:55:00,2016-03-06 03:00:00,0.864583
23,INC0000060,0,0,3,True,Caller 4491,2016-02-29 06:38:00,7/3/2016 13:00,Location 204,Category 9,...,Symptom 450,2 - Medium,2 - Medium,3 - Moderate,Group 25,Resolver 125,True,2016-03-02 12:06:00,2016-03-07 13:00:00,2.227778
31,INC0000062,1,0,7,False,Caller 3765,2016-02-29 06:58:00,5/3/2016 16:00,Location 93,Category 53,...,Symptom 232,1 - High,2 - Medium,2 - High,Group 23,<NA>,True,2016-02-29 15:51:00,2016-03-05 16:00:00,0.370139
39,INC0000063,1,0,7,True,Caller 2146,2016-02-29 07:08:00,5/3/2016 17:00,Location 93,Category 20,...,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 23,<NA>,True,2016-02-29 16:01:00,2016-03-05 17:00:00,0.370139
48,INC0000064,1,0,8,True,Caller 2838,2016-02-29 07:10:00,8/3/2016 17:00,Location 143,Category 53,...,Symptom 580,2 - Medium,2 - Medium,3 - Moderate,Group 28,Resolver 78,True,2016-03-03 16:00:00,2016-03-08 17:00:00,3.368056
61,INC0000065,6,0,13,True,Caller 5323,2016-02-29 07:38:00,7/3/2016 16:00,Location 108,Category 45,...,Symptom 311,2 - Medium,2 - Medium,3 - Moderate,Group 33,Resolver 216,True,2016-03-02 15:21:00,2016-03-07 16:00:00,2.321528
65,INC0000066,1,0,3,True,Caller 3796,2016-02-29 08:03:00,7/3/2016 15:00,Location 161,Category 55,...,<NA>,2 - Medium,2 - Medium,3 - Moderate,Group 54,<NA>,True,2016-03-02 14:37:00,2016-03-07 15:00:00,2.273611
76,INC0000067,1,0,10,True,Caller 442,2016-02-29 08:03:00,7/3/2016 12:00,Location 143,Category 9,...,Symptom 470,2 - Medium,2 - Medium,3 - Moderate,Group 28,Resolver 236,True,2016-03-02 11:11:00,2016-03-07 12:00:00,2.130556


Ahora, tambien es importante revisar si existen datos aberrantes y/o atipicos. 

**Nota:** Para datos atipicos usamos métodos estadísticos como IQR (rango intercuartílico)

In [6]:
# Detectar tiempos de resolución negativos
aberrantes = df_unicos[df_unicos["resolution_time"] < 0]
print("Cantidad de datos aberrantes:", aberrantes.shape[0])



Q1 = df_unicos["resolution_time"].quantile(0.25)
Q3 = df_unicos["resolution_time"].quantile(0.75)
IQR = Q3 - Q1

# Detectar datos atípicos (sobre todo con tiempos de resolución)
atipicos = df_unicos[(df_unicos["resolution_time"] < (Q1 - 1.5 * IQR)) |
              (df_unicos["resolution_time"] > (Q3 + 1.5 * IQR))]

print("Cantidad de datos atípicos:", atipicos.shape[0])


Cantidad de datos aberrantes: 0
Cantidad de datos atípicos: 2536


Por último, se hace revisión de algunas estadisticas descriptivas, especialmente para las **variables numéricas**. También se exportan los datos limpios para posteriormente ser manipulados en la fase de modelamiento.

In [7]:
df.describe()
df.to_excel("datos_limpios.xlsx", index=False)

In [8]:
df_unicos.describe()
df_unicos.to_excel("datos_unicos.xlsx", index=False)